In [ ]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint
from time import sleep
import re
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os
from selenium.common.exceptions import NoSuchElementException

In [ ]:
folderPath = 'ebooks'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

In [ ]:
my_service = Service(executable_path="./chromedriver.exe")
url = 'https://www.gutenberg.org/browse/languages/zh'
count = 0
for i in range(0,400):
    driver = webdriver.Chrome(service=my_service)
    driver.get(url)
    sleep(1)
    language = driver.find_elements(By.CSS_SELECTOR,'div.pgdbbylanguage li.pgdbetext a')[i]
    language.click()
    sleep(1)
    # 抓檔名
    title = driver.find_element(By.CSS_SELECTOR,'div#content h1')
    soupTitle = bs(title.text,'lxml')
    try:
        with open(folderPath + '/' + soupTitle.text +'.txt' ,'a')as file:
            file.write("")

        #抓內文
        regex = r'[\u4E00-\u9FFF]'
        try:
            enter = driver.find_element(By.LINK_TEXT,'Plain Text UTF-8')
            enter.click()
            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located(
                        (By.CSS_SELECTOR,'pre')
                    )
                )
                content = driver.find_element(By.CSS_SELECTOR,'pre')
                chinese_text = "".join(re.findall(regex,content.text))

            except TimeoutException:
                print("等待逾時")

            # 寫成檔案
            with open(folderPath + '/' + soupTitle.text +'.txt' ,'a',encoding='utf-8')as file:
                file.write(chinese_text)
            count += 1    
            if count > 230:
                print("完成了")
                break
        except NoSuchElementException:
            continue
    except FileNotFoundError:
        continue
    finally:
        driver.close()